# TMK Foundation Setup - Appetizer 🥗

**Purpose**: Create the TMK foundation layer that all other data will join to
**Input**: Distance to Municipal Wells (contains TMK data)
**Output**: TMK_Foundation_Master in Outputs/TMK_Foundation/

## Recipe Overview
This is our foundation layer - like preparing the base for a complex dish. Everything else builds on this.

### Ingredients:
- Municipal Wells shapefile (contains TMK and distance data)
- Master table field structure

### Cooking Steps:
1. Analyze TMK data structure
2. Create foundation layer
3. Add fields for subsequent joins
4. Initialize tracking system

In [ ]:
# Load master configuration if not already loaded
if 'outputs_folder' not in globals():
    exec(open(r"../00_Master_Menu/00_Master_Hawaii_Matrix_Analysis.ipynb").read())

import sys
sys.path.append(os.path.join(scripts_folder, '99_Kitchen_Utils'))
from Common_Functions import *

print("🥗 Starting TMK Foundation Setup...")
log_workflow_step("TMK Foundation", "Beginning setup")

In [ ]:
# Create TMK Foundation folder structure
tmk_folder = os.path.join(outputs_folder, "TMK_Foundation")
if not os.path.exists(tmk_folder):
    os.makedirs(tmk_folder)
    print(f"Created: {tmk_folder}")

# Validate source layer exists
validate_layer_exists(municipal_wells_layer)

# Analyze TMK structure
wells_fields = [(f.name, f.type) for f in arcpy.ListFields(municipal_wells_layer)]
wells_count = int(arcpy.management.GetCount(municipal_wells_layer)[0])

print(f"Municipal Wells Analysis:")
print(f"  Records: {wells_count:,}")
print(f"  TMK fields: {[f[0] for f in wells_fields if 'TMK' in f[0].upper()]}")

In [ ]:
# Create TMK Foundation layer
tmk_foundation_name = "TMK_Foundation_Master"
tmk_foundation_path = os.path.join(tmk_folder, tmk_foundation_name + ".shp")

# Copy Municipal Wells as foundation
arcpy.management.CopyFeatures(municipal_wells_layer, tmk_foundation_path)
print(f"✅ Created foundation: {tmk_foundation_path}")

# Add master table tracking fields
foundation_fields = [
    ("SOIL_JOINED", "TEXT", 10, "Soil data joined flag"),
    ("SLOPE_CLASS", "TEXT", 15, "HAR 11-62 Slope Class"),
    ("PERC_CLASS", "TEXT", 20, "HAR 11-62 Percolation Class"),
    ("DRAINAGE_CLASS", "TEXT", 20, "Drainage Suitability"),
    ("SEPTIC_OK", "SHORT", None, "Septic Compatible (1/0)"),
    ("ATU_OK", "SHORT", None, "ATU Compatible (1/0)"),
    ("SEEPAGE_PIT_OK", "SHORT", None, "Seepage Pit Compatible (1/0)"),
    ("JOIN_LOG", "TEXT", 200, "Join sequence tracking")
]

for field_name, field_type, field_length, field_alias in foundation_fields:
    try:
        if field_length:
            arcpy.management.AddField(tmk_foundation_path, field_name, field_type, 
                                   field_length=field_length, field_alias=field_alias)
        else:
            arcpy.management.AddField(tmk_foundation_path, field_name, field_type, 
                                   field_alias=field_alias)
        print(f"  ✅ Added: {field_name}")
    except Exception as e:
        print(f"  ⚠️  Field {field_name}: {e}")

# Initialize tracking
with arcpy.da.UpdateCursor(tmk_foundation_path, ["JOIN_LOG"]) as cursor:
    for row in cursor:
        row[0] = f"Foundation: {datetime.now().strftime('%Y-%m-%d %H:%M')}"
        cursor.updateRow(row)

print(f"✅ TMK Foundation ready for sequential joins!")
log_workflow_step("TMK Foundation", "Setup complete")